In [120]:
import numpy as np
import pandas as pd

#read the given text file, using separator as '|'
df = pd.read_csv('gbmdata02032022.txt', skiprows = 4, sep = '|')

df = df.reset_index(drop = True)
#remove the first unnecessary column 'Unnamed : 0'
df.drop(['Unnamed: 0'], inplace = True, axis = 1)

#remove the extra spaces in the column names
df.columns = [x.strip() for x in df.columns.values]

#packages to read ra and dec 
#convert b/w hrs and deg easily
from astropy.coordinates import SkyCoord, FK4
import astropy.units as u

#make a new column concatenating 'ra' and 'dec'
df['RD'] = df['ra'] + df['dec']

#there are some NaN values in RD columns
#removing all the rows which have NaN in RD column
df = df.dropna(axis = 0, subset = ['RD'])

#remove rows from 3231 to 3233
df = df.drop(df.index[[3231,3232,3233]])
#resetting index
df.reset_index(drop = True, inplace = True)

In [121]:
#numpy array of SkyCoord tuple (ra, dec)
radec = np.array([SkyCoord(x, frame = FK4, unit = (u.hourangle, u.deg), obstime = "J2000") for x in df['RD']])

#add two new columns with ra and dec values 
df['ra_val'] = np.array([x.ra.value for x in radec])
df['dec_val'] = np.array([x.dec.value for x in radec])

In [137]:
R = 10 #radius in degrees

In [149]:
#find nearbyGRB in a 'R' degree radius
#n = index (center of circle)
#ra_val = ra values , dec_val = dec values
def nearbyGRB(n, ra_val, dec_val):
    nbd = []
    ra = ra_val[n]
    dec = dec_val[n]
    for i in range(n+1, len(ra_val)):
        if ra_val[i] == ra or dec_val[i] == dec:
            continue
        if((ra_val[i] - ra)**2 + (dec_val[i] - dec)**2 <= R**2):
            nbd.append((i, ra_val[i], dec_val[i]))
    return nbd

In [150]:
import time 
start = time.time()

#tuple containing all the nearby GRBs
#tuple : (id, ra, dec) 
# index value = center of the circle
# id corresponds to ra/dec

nbd = []
for i in range(len(df['ra_val'])):
    nbd.append(nearbyGRB(i, df['ra_val'], df['dec_val']))

end = time.time()
print('time elapsed : {}'.format(end - start))

time elapsed : 210.140962600708


In [141]:
nbd

[[(3, 48.958749999999995, -87.4786111111111),
  (3232, 48.958749999999995, -87.4786111111111)],
 [],
 [(6, 285.48999999999995, -86.0),
  (15, 296.39, -80.45),
  (3235, 285.48999999999995, -86.0),
  (3244, 296.39, -80.45)],
 [(11, 41.32999999999999, -83.33), (3240, 41.32999999999999, -83.33)],
 [(7, 71.00999999999999, -85.93), (3236, 71.00999999999999, -85.93)],
 [(6, 285.48999999999995, -86.0), (3235, 285.48999999999995, -86.0)],
 [(3231, 294.60999999999996, -87.58),
  (3234, 276.57291666666663, -86.11138888888888)],
 [(31, 66.50625, -77.26472222222222),
  (3233, 72.94, -87.43),
  (3260, 66.50625, -77.26472222222222)],
 [(26, 9.479999999999999, -78.2), (3255, 9.479999999999999, -78.2)],
 [],
 [(12, 188.14999999999998, -82.69),
  (23, 187.49999999999997, -74.0),
  (3241, 188.14999999999998, -82.69),
  (3252, 187.49999999999997, -74.0)],
 [(14, 31.999999999999996, -81.2),
  (25, 34.49999999999999, -78.38),
  (3232, 48.958749999999995, -87.4786111111111),
  (3243, 31.999999999999996, -81.